In [1]:
!nvidia-smi

Mon May 20 10:36:52 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:07:00.0 Off |                    0 |
| N/A   27C    P0    70W / 400W |   5640MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:0A:00.0 Off |                    0 |
| N/A   

In [2]:
import torch


# Function to list available GPUs and select one
def select_device():
    if torch.cuda.is_available():
        print("Available GPUs:")
        for i in range(torch.cuda.device_count()):
            print(f"{i}: {torch.cuda.get_device_name(i)}")
        device_id = int(input("Select GPU by entering the device ID (default 0): ") or 0)
        if device_id < torch.cuda.device_count():
            print(f"Using GPU: {torch.cuda.get_device_name(device_id)}")
            return torch.device(f"cuda:{device_id}")
        else:
            print(f"Invalid device ID. Using GPU: {torch.cuda.get_device_name(0)}")
            return torch.device("cuda:0")
    else:
        print("No GPU available. Using CPU.")
        return torch.device("cpu")

# Select the device
device = select_device()

Available GPUs:
0: NVIDIA A100-SXM4-40GB
1: NVIDIA A100-SXM4-40GB
2: NVIDIA A100-SXM4-40GB
3: NVIDIA A100-SXM4-40GB
4: NVIDIA A100-SXM4-40GB
5: NVIDIA A100-SXM4-40GB
6: NVIDIA A100-SXM4-40GB
7: NVIDIA A100-SXM4-40GB
Select GPU by entering the device ID (default 0): 0
Using GPU: NVIDIA A100-SXM4-40GB


In [3]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from torch.utils.data import DataLoader, Dataset
import torch

# Define the model and tokenizer for RoBERTa large
model_name = "facebook/roberta-large"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)  # Assuming 3 labels for the task



/opt/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


OSError: facebook/roberta-large is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
import pandas as pd

# SNLI train dataset

df_snli_train = pd.read_csv('SNLI/snli_1.0_train.csv')

In [ ]:
df_snli_train

In [ ]:
df_snli_train.info()

In [ ]:
# SNLI test dataset

df_snli_test = pd.read_csv('SNLI/snli_1.0_test.csv')

In [ ]:
df_snli_test

In [ ]:
df_snli_test.info()

In [ ]:
# Step 1: Load and preprocess the data
def load_and_preprocess_data(filepath):
    # Load the dataset
    df = pd.read_csv(filepath)
    
    # Select only the required columns
    df = df[['sentence1', 'sentence2', 'gold_label']]
    
    # Convert sentences to lowercase to standardize the text
    df['sentence1'] = df['sentence1'].str.lower()
    df['sentence2'] = df['sentence2'].str.lower()
    
    # Mapping labels to integers
    label_mapping = {'entailment': 0, 'neutral': 1, 'contradiction': 2}
    df['gold_label'] = df['gold_label'].map(label_mapping)
    
    # Drop rows with any missing values in these columns
    df.dropna(subset=['sentence1', 'sentence2', 'gold_label'], inplace=True)
    
    # Ensure the gold_label column is of type integer
    df['gold_label'] = df['gold_label'].astype(int)
    
    return df

# File paths
train_path = 'SNLI/snli_1.0_train.csv'
dev_path = 'SNLI/snli_1.0_dev.csv'

# Load and preprocess the training and development datasets
df_snli_train = load_and_preprocess_data(train_path)
df_snli_dev = load_and_preprocess_data(dev_path)

# Display the first few rows of the training data to verify
print(df_snli_train.head())


In [ ]:
df_snli_dev.head()

In [ ]:
import matplotlib.pyplot as plt

# Plotting the distribution of labels
label_counts = df_snli_train['gold_label'].value_counts()
plt.figure(figsize=(8, 5))
label_counts.plot(kind='bar')
plt.title('Distribution of Labels in the Training Data')
plt.xlabel('Labels')
plt.ylabel('Frequency')
plt.xticks(rotation=0)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Function to calculate the length of tokenized sentences
def calculate_lengths(tokenizer, sentences):
    lengths = []
    for sentence in sentences:
        # Tokenize and count the number of tokens
        tokens = tokenizer.tokenize(sentence)
        lengths.append(len(tokens))
    return lengths

# Concatenate the sentences from both fields for a more comprehensive overview
all_sentences = df_snli_train['sentence1'].tolist() + df_snli_train['sentence2'].tolist()

# Calculate lengths
lengths = calculate_lengths(tokenizer, all_sentences)

# Plot histogram
plt.figure(figsize=(10, 5))
plt.hist(lengths, bins=range(0, max(lengths) + 10, 5), alpha=0.75, color='blue')
plt.title('Distribution of Sentence Lengths')
plt.xlabel('Length of tokenized sentences')
plt.ylabel('Number of sentences')
plt.grid(True)
plt.show()


In [ ]:
# Optional: Plot distribution of labels to check balance
import matplotlib.pyplot as plt
label_counts = df_snli_dev['gold_label'].value_counts()
plt.figure(figsize=(8, 5))
label_counts.plot(kind='bar')
plt.title('Distribution of Labels in the Validation Data')
plt.xlabel('Labels')
plt.ylabel('Frequency')
plt.xticks(rotation=0)
plt.show()

In [ ]:
import logging

logging.disable(logging.WARNING)



def tokenize_and_prepare_inputs(df, tokenizer, max_length=45):
    # Tokenize, pad and create attention masks
    # 'max_length' is a parameter you can tune depending on the maximum length you observed in your dataset analysis
    tokenized_inputs = tokenizer(
        df['sentence1'].tolist(),
        df['sentence2'].tolist(),
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )
    
    # Extracting input_ids and attention masks from the tokenized inputs
    input_ids = tokenized_inputs['input_ids']
    attention_masks = tokenized_inputs['attention_mask']
    
    return input_ids, attention_masks

# Tokenize training data
train_input_ids, train_attention_masks = tokenize_and_prepare_inputs(df_snli_train, tokenizer)

# Tokenize validation data
validation_input_ids, validation_attention_masks = tokenize_and_prepare_inputs(df_snli_dev, tokenizer)

# Convert labels to tensors
train_labels = torch.tensor(df_snli_train['gold_label'].values)
validation_labels = torch.tensor(df_snli_dev['gold_label'].values)

# Print some details to verify everything is as expected
print("Training input IDs shape:", train_input_ids.shape)
print("Validation input IDs shape:", validation_input_ids.shape)
print("Training labels shape:", train_labels.shape)
print("Validation labels shape:", validation_labels.shape)


In [ ]:
from torch.utils.data import Dataset, DataLoader

class SNLIDataset(Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': self.labels[idx]
        }
        return item

    
train_dataset = SNLIDataset(train_input_ids, train_attention_masks, train_labels)
validation_dataset = SNLIDataset(validation_input_ids, validation_attention_masks, validation_labels)

batch_size = 32  # Adjust based on your GPU memory

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
import torch
from torch.optim import AdamW 
from torch.optim.lr_scheduler import CyclicLR
from tqdm.auto import tqdm




model.to(device)

# Optimizer setup
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=1e-6)

# CLR Scheduler setup (Ensure only one scheduler setup is active)
scheduler = CyclicLR(optimizer, base_lr=5e-6, max_lr=1e-5, 
                     step_size_up=len(train_loader)//2, step_size_down=len(train_loader)//2,
                     mode='triangular', cycle_momentum=False)

# Training and validation functions
def train_model(model, train_loader, validation_loader, optimizer, scheduler, epochs=3, save_path='model_snapshots'):
    steps_per_epoch = len(train_loader)
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}', unit='batch')

        for batch_index, batch in enumerate(progress_bar):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()  # Update the learning rate after each batch

            train_loss += loss.item()
            _, predicted = torch.max(logits, dim=1)
            correct += (predicted == batch['labels']).sum().item()
            total += batch['labels'].size(0)
            accuracy = 100. * correct / total
            lr = scheduler.get_last_lr()[0]

            progress_bar.set_postfix(loss=train_loss/(batch_index + 1), accuracy=f'{accuracy:.2f}%', lr=f'{lr:.6f}')

            # Save snapshot at the end of each cycle
            current_step = epoch * steps_per_epoch + batch_index
            if (current_step + 1) % scheduler.total_size == 0:
                torch.save(model.state_dict(), f'{save_path}/roberta_large_cycle_{current_step + 1}.pth')
                print(f"Snapshot saved at step {current_step + 1}")

        validate_model(model, validation_loader, device)

def validate_model(model, validation_loader, device):
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in tqdm(validation_loader, desc='Validating', leave=False, unit='batch'):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits
            val_loss += loss.item()
            _, predicted = torch.max(logits, dim=1)
            correct += (predicted == batch['labels']).sum().item()
            total += batch['labels'].size(0)

    val_accuracy = 100. * correct / total
    print(f'\nValidation Loss: {val_loss / len(validation_loader):.4f} | Validation Accuracy: {val_accuracy:.2f}%')

# Start training
train_model(model, train_loader, validation_loader, optimizer, scheduler, epochs=5, save_path='/storage/data/st1070263')